In [ ]:
%matplotlib qt

import numpy as np
from copy import deepcopy
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from itertools import product

class Sudoku:
    def __init__(self, filename = None):
        self.board = None
        self.swappable = None
        self.swappable_len = None
        if filename is not None:
            self.read_sudoku(filename)
    
    def init_solution(self):
        for row_off in range(3):
            for col_off in range(3):
                given = set(sum(list(row[3*col_off:3*col_off+3] for row in self.board[3*row_off:3*row_off+3]), []))
                act = 1
                for (row, col) in self.swappable[row_off][col_off]:
                    while act in given:
                        act += 1
                    self.board[3*row_off + row][3*col_off + col] = act
                    act += 1
    
    def read_sudoku(self, filename):
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        self.swappable = [[[(int(i/3), i % 3) for i in range(9)] for _ in range(3)] for _ in range(3)] 
        self.swappable_len = [[9 for _ in range(3)] for _ in range(3)] 
        with open(filename, 'r') as file:
            for (row, line) in enumerate(file.readlines()):
                for (col, sign) in enumerate(line[:-1].split(sep = " ")):
                    if sign.isdigit():
                        box_r = int(row/3)
                        box_c = int(col/3)
                        self.swappable[box_r][box_c].remove((row % 3, col % 3))
                        self.swappable_len[box_r][box_c] -= 1
                        self.board[row][col] = int(sign)
    
    def hide_random(self, n):
        ll = [t for t in product(range(9), repeat=2)]
        np.random.shuffle(ll)
        
        for (row, col) in ll[:n+1]:
            box_r = int(row/3)
            box_c = int(col/3)
            self.swappable[box_r][box_c].append((row % 3, col % 3))
            self.swappable_len[box_r][box_c] += 1
            self.board[row][col] = 0
            
    def copy(self):
        ret = Sudoku()
        ret.board = deepcopy(self.board)
        ret.swappable = self.swappable
        ret.swappable_len = self.swappable_len
        return ret
                    
    def energy(self):
        E = 0
        for i in range(9):
            seen = set()
            for j in range(9):
                num = self.board[i][j]
                if num in seen:
                    E += 1
                else:
                    seen.add(num)
            E += 9 - len(seen)
                    
        for i in range(9):
            seen = set()
            for j in range(9):
                num = self.board[j][i]
                if num in seen:
                    E += 1
                else:
                    seen.add(num)
            E += 9 - len(seen)
            
        return E
            
    def swap(self, row1, col1, row2, col2):
        self.board[row2][col2], self.board[row1][col1] = self.board[row1][col1], self.board[row2][col2] 
    
    def rand_swap(self):
        row_offset = np.random.randint(0, 3)
        col_offset = np.random.randint(0, 3)
        
        while(len(self.swappable[row_offset][col_offset]) < 2):
            row_offset = np.random.randint(0, 3)
            col_offset = np.random.randint(0, 3)
        
        index = np.random.randint(0, self.swappable_len[row_offset][col_offset])
        place1 = self.swappable[row_offset][col_offset][index]
        place2 = self.swappable[row_offset][col_offset][index - np.random.randint(1, self.swappable_len[row_offset][col_offset])]
        self.swap(3*row_offset + place1[0], 3*col_offset + place1[1], 3*row_offset + place2[0], 3*col_offset + place2[1])
            
    def print_row(self, i):
        for j in range(9):
            print(self.board[i][j], end=" ")
            if (j + 1) % 3 == 0:
                print("\b|", end="")
        print("\b")
            
    def print_sudoku(self):
        for row in range(9):
            self.print_row(row)
            if (row + 1) % 3 == 0 and row < 8:
                print("-"*5 + "+" + "-"*5 + "+" + "-"*5)
                
    def sa_solve(self, Tmax = 0.7, Tmin = 0.2):
        T = Tmax
        starting_sudoku = self.copy()
        self.init_solution()
        current_energy = self.energy()
        best_energy = current_energy
        
        energies = [current_energy]
        while T > Tmin and best_energy != 0:
            best_board = self.board
            best_energy = current_energy
            
            cand = self.copy()
            for _ in range(np.random.randint(1, 4)):
                cand.rand_swap()
            
            cand_energy = cand.energy()
            energy_delta = cand_energy - best_energy
            
            if energy_delta <= 0 or bernoulli.rvs(p=np.exp(-energy_delta/T)) == 1:
                best_energy = cand_energy
                best_board = cand.board
            
                
            self.board = best_board
            current_energy = best_energy
            
            if current_energy == 2:
                self.print_sudoku()
            
            energies.append(current_energy)
            
            T *= 0.99999
            
        return energies
        

# Przykładowe sudoku

In [ ]:
s_easy = Sudoku("easy_sudoku")
s_easy.print_sudoku()
print()
en = s_easy.sa_solve()
s_easy.print_sudoku()
print(f"repeats = {en[-1]}")

In [ ]:
s_med = Sudoku("med_sudoku")
s_med.print_sudoku()
print()
en = s_med.sa_solve()
s_med.print_sudoku()
print(f"repeats = {en[-1]}")

In [ ]:
s_hard = Sudoku("hard_sudoku")
s_hard.print_sudoku()
print()
en = s_hard.sa_solve()
s_hard.print_sudoku()
print(f"repeats = {en[-1]}")

In [ ]:
def avg_iters_to_solve(sudoku, tries):
    iters = []
    unsolved = 0
    solved = 0
    while solved != tries:
        cp_sudoku = sudoku.copy()
        energies = cp_sudoku.sa_solve()
        if energies[-1] != 0:
            unsolved += 1
        else:
            solved += 1
            iters.append(len(energies))
    print(f"Not solved {unsolved} times")
    return sum(iters)/tries

# Ilość iteracji potrzebna do znalezienia rozwiązania w zależności od ilości pustych miejsc

In [ ]:
iters_plot_x = []
iters_plot_y = []
for n in range(11, 82, 2):
    s_tmp = Sudoku("filled_sudoku")
    s_tmp.hide_random(n)
    
    print(n)
    iters_plot_x.append(n)
    iters_plot_y.append(avg_iters_to_solve(s_tmp, 3))
    
plt.plot(iters_plot_x, iters_plot_y)
plt.show()